# Scraping Data that uses Javascript - Nike.com

In [1]:
import time
import pandas as pd

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys


In [2]:
# Declare Selenium application
service = Service('C:/Users/user/OneDrive - 4y0flc/DataScience/Selenium-ChromeDriver/chromedriver.exe')

In [3]:
# Launch new chrome browser.
driver = webdriver.Chrome(service=service)
driver.maximize_window()
driver.get('https://www.google.com.sg/')

# find input element
box = driver.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input')
box.send_keys('nike')
box.send_keys(Keys.ENTER)

time.sleep(3)

# Click first result
driver.find_element('xpath','//*[@id="rso"]/div[1]/div/div/div/div/div/div/div/div[1]/a/h3').click()

time.sleep(2)

# Find sale button
driver.find_element('xpath','/html/body/div[3]/div/div[3]/header/div/div[1]/div[2]/nav/div[2]/ul/li[5]/a').click()

time.sleep(3)

# Get current Height
last_height = driver.execute_script('return document.body.scrollHeight')

# Scroll through all the pages
while True:
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    time.sleep(3)
    new_height = driver.execute_script('return document.body.scrollHeight')
    if new_height == last_height:
        break
    last_height = new_height
#

In [4]:
#Imports the HTML of the webpage into python  
soup = BeautifulSoup(driver.page_source, 'lxml')

In [5]:
#grabs the HTML of each product
product_card = soup.find_all('div', class_ = 'product-card__body')
len(product_card)

563

In [ ]:
#Test cell

link = soup.find('a', class_ = 'product-card__link-overlay').get('href')
full_price = soup.find('div', class_ = 'product-price is--striked-out css-0').text.replace('S$', '')
sale_price = soup.find('div', class_ = 'product-price is--current-price css-1ydfahe').text.replace('S$', '')


print(link)
print(full_price)
print(sale_price)

Discount = ((float(full_price) - float(sale_price)) / float(full_price)) * 100
print(Discount)
DiscountPct = "{:.2%}".format(((float(full_price) - float(sale_price)) / float(full_price))) 
print(DiscountPct)
print ( "{:.2%}".format(((float(full_price) - float(sale_price)) / float(full_price))) )


In [6]:
#Creates a dataframe using the column headers from our table
df = pd.DataFrame( {'Link':[''], 'Name':[''], 'Subtitle':[''], 'Price':[''], 'Sale Price':[''] } )
df

,Link,Name,Subtitle,Price,Sale Price
0,,,,,


In [7]:
# Scrape every link, name, subtitle, full price, sale price
for product in product_card:
    try:
        link = product.find('a', class_ = 'product-card__link-overlay').get('href')
        name = product.find('div', class_ = 'product-card__title').text
        subtitle = product.find('div', class_ = 'product-card__subtitle').text.replace('S$', '')
        full_price = product.find('div', class_ = 'product-price is--striked-out css-0').text.replace('S$', '')
        sale_price = product.find('div', class_ = 'product-price is--current-price css-1ydfahe').text.replace('S$', '')

        df = pd.concat([df, pd.DataFrame.from_records([{ 'Link': link,'Name': name, 'Subtitle': subtitle,'Price': full_price, 'Sale Price': sale_price }])], ignore_index=True)
    except:
        pass

In [8]:
# Check number of result
len(df)

564

In [9]:
df.head()

,Link,Name,Subtitle,Price,Sale Price
0,,,,,
1,https://www.nike.com/sg/t/space-hippie-04-shoe...,Nike Space Hippie 04,Women's Shoes,229,179.90
2,https://www.nike.com/sg/t/air-force-1-lv8-olde...,Nike Air Force 1 LV8,Older Kids' Shoes,129,99.90
3,https://www.nike.com/sg/t/air-zoom-tempo-next-...,Nike Air Zoom Tempo NEXT%,Men's Road Running Shoes,279,219.90
4,https://www.nike.com/sg/t/react-infinity-run-f...,Nike React Infinity Run Flyknit 3,Men's Road Running Shoes,235,189.90


In [10]:
# Remove 1st row
df = df.iloc[1: , :]
df.head()

,Link,Name,Subtitle,Price,Sale Price
1,https://www.nike.com/sg/t/space-hippie-04-shoe...,Nike Space Hippie 04,Women's Shoes,229,179.90
2,https://www.nike.com/sg/t/air-force-1-lv8-olde...,Nike Air Force 1 LV8,Older Kids' Shoes,129,99.90
3,https://www.nike.com/sg/t/air-zoom-tempo-next-...,Nike Air Zoom Tempo NEXT%,Men's Road Running Shoes,279,219.90
4,https://www.nike.com/sg/t/react-infinity-run-f...,Nike React Infinity Run Flyknit 3,Men's Road Running Shoes,235,189.90
5,https://www.nike.com/sg/t/dunk-high-retro-shoe...,Nike Dunk High Retro,Men's Shoe,179,139.90


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 563 entries, 1 to 563
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Link        563 non-null    object
 1   Name        563 non-null    object
 2   Subtitle    563 non-null    object
 3   Price       563 non-null    object
 4   Sale Price  563 non-null    object
dtypes: object(5)
memory usage: 22.1+ KB


In [12]:
# Save to CSV
df.to_csv('Nike.csv', index=False)

In [13]:
# Read the CSV file
df = pd.read_csv('Nike.csv')
df.head()

,Link,Name,Subtitle,Price,Sale Price
0,https://www.nike.com/sg/t/space-hippie-04-shoe...,Nike Space Hippie 04,Women's Shoes,229,179.9
1,https://www.nike.com/sg/t/air-force-1-lv8-olde...,Nike Air Force 1 LV8,Older Kids' Shoes,129,99.9
2,https://www.nike.com/sg/t/air-zoom-tempo-next-...,Nike Air Zoom Tempo NEXT%,Men's Road Running Shoes,279,219.9
3,https://www.nike.com/sg/t/react-infinity-run-f...,Nike React Infinity Run Flyknit 3,Men's Road Running Shoes,235,189.9
4,https://www.nike.com/sg/t/dunk-high-retro-shoe...,Nike Dunk High Retro,Men's Shoe,179,139.9


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 563 entries, 0 to 562
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Link        563 non-null    object 
 1   Name        563 non-null    object 
 2   Subtitle    563 non-null    object 
 3   Price       563 non-null    int64  
 4   Sale Price  563 non-null    float64
dtypes: float64(1), int64(1), object(3)
memory usage: 22.1+ KB


In [15]:
# add a new column
df['PercentDiff'] = (df['Price'] - df['Sale Price'] ) / df['Price'] * 100
df.head()

,Link,Name,Subtitle,Price,Sale Price,PercentDiff
0,https://www.nike.com/sg/t/space-hippie-04-shoe...,Nike Space Hippie 04,Women's Shoes,229,179.9,21.441048
1,https://www.nike.com/sg/t/air-force-1-lv8-olde...,Nike Air Force 1 LV8,Older Kids' Shoes,129,99.9,22.558140
2,https://www.nike.com/sg/t/air-zoom-tempo-next-...,Nike Air Zoom Tempo NEXT%,Men's Road Running Shoes,279,219.9,21.182796
3,https://www.nike.com/sg/t/react-infinity-run-f...,Nike React Infinity Run Flyknit 3,Men's Road Running Shoes,235,189.9,19.191489
4,https://www.nike.com/sg/t/dunk-high-retro-shoe...,Nike Dunk High Retro,Men's Shoe,179,139.9,21.843575


In [18]:
# Save to CSV
df.to_csv('Nike.csv', index=False)